In [ ]:
get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt
from urllib.request import Request, urlopen
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
from torch.utils import data
import torch.optim as optim
import numpy as np
import os, sys
import glob
from tensorboardX import SummaryWriter
import pickle
from progressbar import ProgressBar

In [ ]:
test_epochs = 1
uniprot_xs = 7000

In [ ]:
os.system('tensorboard --logdir runs &')

In [ ]:
# example usage
#a = torch.cuda.FloatTensor(4, 4, 1) # create tensor on gpu
#b = a.cpu() --- transfer a to cpu
#c = b.numpy() --- turn into numpy array

In [ ]:
params = {}
params['num_layers'] = []
params['filter_sz'] = []
params['num_filters'] = []
params['learning_rate'] = []
params['batch_sz'] = []
params['epochs'] = []
params['loss_last'] = []
params['acc_last']= []
params['lr_step'] = []
params['lr_decay'] = []

In [ ]:
def loss_acc(pred, label):
    l = loss_func(pred, label).cuda()
    pr = pred.data.cpu().numpy()
    la = label.data.cpu().numpy()
    a = np.mean(np.int32(np.equal(np.argmax(pr, 1), la)))
    return l, a

In [ ]:
def plot(loss, testloss, acc, testacc, num):
    fig = plt.figure(figsize=(10, 5))
    a = fig.add_subplot(121)
    a.plot(loss)
    a.plot(np.arange(len(testloss))*num, testloss)
    a.set_xlabel('Training Iteration')
    a.set_ylabel('Loss')
    b = fig.add_subplot(122)
    b.plot(acc)
    b.plot(np.arange(len(testacc))*num, testacc)
    b.set_xlabel('Training Iteration')
    b.set_ylabel('Accuracy')
    plt.show()

In [ ]:
def update_param_dict(nl, fs, nf, lr, bs, ep, lrs, lrd):
    params['num_layers'].append(nl)
    params['filter_sz'].append(fs)
    params['num_filters'].append(nf)
    params['learning_rate'].append(lr)
    params['batch_sz'].append(bs)
    params['epochs'].append(ep)
    params['lr_step'].append(lrs)
    params['lr_decay'].append(lrd)

In [ ]:
def prepare_batch(x, y):
    x = torch.from_numpy(x).cuda().float()
    y = torch.from_numpy(y).cuda().long()
    return Variable(x), Variable(y)

In [ ]:
def get_uniprot_data(kw, numxs):
    '''Goes to the uniprot website and searches for 
       data with the keyword given. Returns the data 
       found up to limit elements.'''

    kws = [kw, 'NOT+' + kw]
    Protein_data = {}
            
    for i in range(2):
        kw = kws[i]
        url1 = 'http://www.uniprot.org/uniprot/?query='
        url2 = '&columns=sequence&format=tab&limit='+str(numxs)
        query_complete = url1 + kw + url2
        request = Request(query_complete)
        response = urlopen(request)
        data = response.read()
        data = str(data, 'utf-8')
        data = data.split('\n')
        data = data[1:-1]
        Protein_data[str(i)] = list(map(lambda x:x.lower(),data))

    x = Protein_data['0'] + Protein_data['1']
    y = np.zeros([numxs*2, ])
    y[:numxs] = 1.
        
    return x, y

In [ ]:
def process_strings(c):
    longest = len(max(c, key=len))
    digits = len(str(longest))
    pad_num = np.ceil(longest*10**(-(digits-2)))
    pad_num = int(pad_num * 10**(digits-2))
    N = len(c)
    X = np.zeros([N, 1, pad_num, 1])
    m = 0
            
    for seq in c:
        x = [] 
        for letter in seq:
            x.append(max(ord(letter)-97, 0))
                    
        x = np.asarray(x)
        diff = pad_num - x.size

        if diff % 2 == 0:
            x = np.pad(x, diff//2, 
                        'constant', constant_values=22.)
        else:
            x = np.pad(x, (int(np.floor(diff/2)), 
                        int(np.ceil(diff/2))), 
                        'constant', constant_values=22.)
        
        X[m, ...] = x[None, :, None]
        m += 1
        
    return X

## Describe Network

In [ ]:
class CNN(nn.Module):
    def __init__(self, n_layers, fsize, num_filters, str_shp):
        super(CNN, self).__init__()
        self.model_ops = {}
        self.n_layers = n_layers 
        ch = 1
        
        for i in range(n_layers):
            if i % 2 == 0 and i > 0:
                num_filters *= 2
            if i == 0: strides=2
            else: strides=1
                
            conv = nn.Conv2d(ch,
                             num_filters,
                             (1, fsize),
                             padding=(0, int(np.floor(fsize/2))),
                             stride=(1, strides)).cuda()
            self.model_ops['conv'+str(i)] = nn.DataParallel(conv)
            ch = num_filters
        #self.fc1 = nn.DataParallel(nn.Linear(ch, 2).cuda())
        self.fc1 = nn.DataParallel(nn.Linear(ch*str_shp//2, 500).cuda())
        self.fc2 = nn.DataParallel(nn.Linear(500, 2).cuda())
        self.sm = nn.DataParallel(nn.Softmax(dim=1).cuda())
        #self.fc2 = nn.DataParallel(nn.Linear(500, 2).cuda())
        
        
    def forward(self, net):
        for j in range(self.n_layers):
            net = self.model_ops['conv'+str(j)](net)
        #net = F.max_pool2d(net, kernel_size=net.size()[2:])
        net = net.view(net.shape[0], -1).cuda(1)
        net = self.fc1(net)
        net = self.fc2(net)
        return self.sm(net)

## Pull data from uniprot and make labels

In [ ]:
X, Y = get_uniprot_data('homeobox', uniprot_xs)
X = process_strings(X)
X = np.transpose(X, (0, 3, 1, 2))
assert(X.shape[0]==Y.shape[0]),'diff. nums of data & labels'

# shuffle data
randtrain = np.random.randint(0, X.shape[0], X.shape[0])
X = X[randtrain, ...]
Y = Y[randtrain]

# split data into training and testing
train_num = int(0.8 * X.shape[0])
testX = X[train_num:, ...]
testY = Y[train_num:]
X = X[:train_num, ...]
Y = Y[:train_num]

print('Train num: %d; Test num: %d'%(X.shape[0], testX.shape[0]))

In [ ]:
for i in range(41):
    Bar = ProgressBar() # initialize progressbar 
    writer = SummaryWriter('runs/train' + str(i))
    writer2 = SummaryWriter('runs/test' + str(i))
    
    
    # each iter use random point in parameter space
    layers = int(np.random.randint(1, 8, 1)[0])
    scale = np.random.uniform(.1, 10.) / np.random.randint(1, 1000, 1)[0]
    learning_rate = np.absolute(np.random.randn(1)[0]) * scale
    batch_sz = np.around(np.random.randint(10, 151, 1)[0])
    fsizes = int(np.random.randint(2, 101, 1)[0])
    if fsizes % 2 == 0: fsizes += 1
    nf = int(np.random.randint(50, 351, 1)[0]) // (layers + 1)
    epochs = int(np.random.randint(8, 61, 1)[0])
    lr_step = np.random.randint(0, epochs, 1)[0]
    lr_d  = np.random.uniform(1e-4, 1.)
    update_param_dict(layers, fsizes, nf, learning_rate,
                      batch_sz, epochs, lr_step, lr_d)
    print(params)
    
    
    # instantiate the network
    Network = CNN(layers, fsizes, nf, X.shape[-1])
    
    # define loss function
    loss_func = nn.CrossEntropyLoss().cuda(1)
    
    # define autodiff optimizer and initialize loss/acc lists
    opt = optim.Adam(Network.parameters(), lr=learning_rate)
    lr_decay = optim.lr_scheduler.StepLR(opt, step_size=lr_step, gamma=lr_d)
    loss_ = []
    acc_ = []
    tloss_ = []
    tacc_ = []
    n_iters = int(np.ceil(X.shape[0] / batch_sz))
    
    for epoch in Bar(range(epochs)):
        # shuffle data and labels
        rand = np.random.permutation(X.shape[0])
        X = X[rand, ...]
        Y = Y[rand]
        
        for iters in range(X.shape[0]//batch_sz):
            if batch_sz*(iters+1) < X.shape[0]:
                x = X[iters*batch_sz:batch_sz*(iters+1), ...]
                y = Y[iters*batch_sz:batch_sz*(iters+1)]
            else:
                x = X[iters*batch_sz:, ...]
                y = Y[iters*batch_sz:]
                
            x, y = prepare_batch(x, y)
        
            # get the output of the network and compute loss/acc.
            loss, acc = loss_acc(Network(x), y)
            acc_.append(acc)
            loss_.append(loss.data[0])
            
            # perform a step down the gradient
            opt.zero_grad() # zero gradient
            loss.backward() # accumulate gradient 
            lr_decay.step() # for lr decay #opt.step()
            
            writer.add_scalar('Train Acc.', acc, epoch*n_iters+iters)
            writer2.add_scalar('Train Loss', loss.data[0], epoch*n_iters+iters)

            
        # test on validation set and plot loss/acc
        if epoch % test_epochs == 0:
            randtest = np.random.randint(0, testX.shape[0], 500)
            tx, ty = prepare_batch(testX[randtest, ...], testY[randtest])
            test_loss, test_acc = loss_acc(Network(tx), ty)
            tloss_.append(test_loss.data[0])
            tacc_.append(test_acc)
            writer.add_scalar('Val. Acc.', test_acc, n_iters*epoch)
            writer2.add_scalar('Val. Loss', test_loss.data[0], n_iters*epoch)
            
    tloss_ = np.mean(np.asarray(tloss_)[-6:])
    tacc_ = np.mean(np.asarray(tacc_)[-6:])
    params['loss_last'].append(tloss_)
    params['acc_last'].append(tacc_)
    #writer.close()
    #writer2.close()

In [ ]:
with open('protein_hparams.pickle', 'wb') as handle:
    pickle.dump(params, handle, protocol=pickle.HIGHEST_PROTOCOL)